In [1]:
from langchain_openai import ChatOpenAI
import os
from openai import api_key, base_url
from transformers import Qwen2_5_VLConfig

from langchain_core.output_parsers.string import StrOutputParser
from langchain.schema import output_parser
# llm = ChatOpenAI(model=)
# OPENAI_API_KEY = os.getenv("OPENAI_API_KEY", "sk-C3gsaWJaOsmjrNv1fIobc4zEEsEkD15T98j80Vc8mNuSE420")
# llm = ChatOpenAI(model="o3-mini", api_key=OPENAI_API_KEY, base_url="https://api.deepbricks.ai/v1/")
QWEN_API_KEY = os.getenv("QWEN_API_KEY", "sk-ypjnechwutigvbyglbkguukzsmzkkxfibauydwkbjrypwojd")
model = ChatOpenAI(model="Qwen/Qwen3-30B-A3B-Instruct-2507", api_key=QWEN_API_KEY, base_url="https://api.siliconflow.cn/v1")



output_parser = StrOutputParser()

/home/zpc/anaconda3/envs/langchain/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
##RAG
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings.huggingface import HuggingFaceEmbeddings

embedding_path = "/home/zpc/langchain/blibli/bge-large-zh-v1.5"
embeddings = HuggingFaceEmbeddings(model_name=embedding_path)



/tmp/ipykernel_144345/4018669994.py:6: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name=embedding_path)


In [3]:
vectorstore = FAISS.from_texts(
    ["小明在华为工作", "熊喜欢吃蜂蜜"],
    embedding=embeddings
)

vectorstore

In [4]:
retriver = vectorstore.as_retriever()
retriver.invoke("熊喜欢吃什么？")

[Document(id='d763e52e-f521-4d01-af84-fc2706869c56', metadata={}, page_content='熊喜欢吃蜂蜜'),
 Document(id='6ca00a0f-56c5-4001-8a91-2d403b1e8c4b', metadata={}, page_content='小明在华为工作')]

In [12]:
from langchain_core.prompts import ChatPromptTemplate
template = """
只根据以下文档回答问题
{context}
问题：{question}"""
prompt = ChatPromptTemplate.from_template(template)

In [16]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

outputParser = StrOutputParser()
setup_and_retrival = RunnableParallel({
    "context":retriver,
    "question": RunnablePassthrough()}
)

chain = setup_and_retrival | prompt | model | outputParser




In [17]:
chain.invoke("小明在哪里工作")

'小明在华为工作。'

In [20]:
from operator import itemgetter
template = """
只根据以下文档回答
{context}
问题: {question},
回答问题请带上称呼:{name}
"""

prompt = ChatPromptTemplate.from_template(template)

chain = (
    {
        "context": itemgetter("question") | retriver,
        "question": itemgetter("question"),
        "name": itemgetter("name"),
    }
    | prompt
    | model
    | output_parser
)



chain.invoke({"question": "小明在哪里工作", "name": "吴迪哥"})


'吴迪哥，小明在华为工作。'

In [23]:
from langchain_community.document_loaders import TextLoader
loader =  TextLoader("/home/zpc/langchain/blibli/data/faq_1.txt")
doc = loader.load()
doc

[Document(metadata={'source': '/home/zpc/langchain/blibli/data/faq_1.txt'}, page_content='一、什么是0分期利息\n\n您好，“0分期利息”是指买家使用花呗、招行掌上生活、工行信用卡、银联信用卡等其他分期购物时无需支付分期利息的功能，分期利息由华为商城承担。\n\n注：自2023年起，商城将相关宣传将“免息”调整为“0分期利息”，主要基于中国银保监会、中国人民银行《关于进一步促进信用卡业务规范健康发展的通知》（银保监规〔2022〕13号），要求“银行业金融机构应当在分期业务合同（协议）首页和业务办理页面以明显方式展示分期业务可能产生的所有息费项目、年化利率水平和息费计算方式。向客户展示分期业务收取的资金使用成本时，应当统一采用利息形式，并明确相应的计息规则，不得采用手续费等形式，法律法规另有规定的除外。”\n\n\n\n二、可以参与0分期利息活动的商品\n\n商城目前仅支持部分单品参与0分期利息，若多商品（含不支持0分期利息）合并支付则不支持0分期利息，以支付页面为准，后续会逐渐开放更多商品，敬请关注。\n\n\n\n三、确认订单分期成功\n\n订单提交成功后在支付方式页面选择分期支付，点选显示0分期利息的支付方式及具体0分期利息期数后，完成支付。\n\n\n\n\n\n四、订单中有多个商品，其中有商品支持0分期利息，为什么提交后却没有0分期利息？\n\n0分期利息商品不能和其它商品一起购买，如果和其他商品购买而不能享用0分期利息，建议取消原来的订单，重新购买时把0分期利息商品和其他商品分开单独购买；且0分期利息的分期数是固定的，如6期0分期利息，并不是3/6/12都提供0分期利息的。\n\n\n\n五、小程序是否支持0分期利息？\n\n华为商城小程序暂不支持0分期利息。\n')]

In [24]:
loader1 = TextLoader("/home/zpc/langchain/blibli/data/faq_2.txt")
doc1 = loader1.load()
doc1

[Document(metadata={'source': '/home/zpc/langchain/blibli/data/faq_2.txt'}, page_content='众测活动\n\n整体介绍：\n\n一、活动定义：众测是以低价试销的形式，通过收集评价、销量等方法，用于测试市场对新商品的反应，用于及时优化销售策略和引导商家改进。\n\n二、优势：众测价通常比较优惠，以不高于大促促销价为原则，最终以与物权方谈判结果为准。\n\n三、适用范围：华为商城所有产品。\n\n\n\n参与方式：\n\n1、华为商城众测的入口在哪里？\n\n华为商城搜索“众测”，即可看到众测入口，点击进入即可；\n\n2、众测上新频次：\n\n众测频道每周一至周五不定期更新上架，部分商品可能会特别调整；\n\n3、众测活动时间：\n\n一款产品众测时间通常为10-20天（热销的商品可能会延期5-10天）。\n\n\n\n常见问题：\n\n1、众测商品的来源?\n\n     您好，众测商品主要来源为华为商城上架的新品，热销的爆款商品也会不定期通过众测回归上线。\n\n2、众测商品的价格会优惠吗？\n\n     您好，众测商品价格对标618和双十一，一般都是该商品某段时间的最低促销价。\n\n3、 众测商品质量会不稳定么？\n\n     您好，众测商品也是量产的正品新品，质量与正式上架商品一致。\n\n4、众测商品买下后多久发货？\n\n     您好，请以商品页显示为准。\n\n5、提交活动订单后多久内支付？\n\n     您好，提交订单后最长付款时效为24小时，逾期订单自动取消\n\n6、成功下单后怎么查询众测订单？\n\n     您好，成功下单后，您可以通过华为商城手机APP、华为商城手机WAP版、华为商城电脑网页版任意一端登录下单账户，在“我的订单”查询。（众测商品订单查询方式等同于正常商品订单）\n\n7、订单支付后未发货前可以取消订单吗？\n\n     您好，在发货前消费者可以取消订单。\n\n8、取消订单后多久内退回款项？\n\n     您好，和华为商城智能家居生态产品通用退款时效一致，3-5个工作日。\n\n9、一个账号可以参与几次众测活动？有限购吗？\n\n     您好，一个账号参与众测活动的次数无限制，没有限购。但每次订单只能购买一个商品。如您需要购买

In [26]:
loader3 = TextLoader("/home/zpc/langchain/blibli/data/yjhx.md", encoding="utf-8")
doc3 = loader3.load()
doc3

[Document(metadata={'source': '/home/zpc/langchain/blibli/data/yjhx.md'}, page_content='**一、选择回收物品**\n\n点击VMALL您可以在设备回收分类中选择旧商品的类别以及品牌，现在VMALL支持回收部分品牌和型号的手机、笔记本、平板、智能数码以及摄影摄像。如若在此页面上找不到您的品牌或型号，您可以点击品牌栏右下方的加号（如下图红圈部分所示）查看VMALL所支持回收的所有品牌型号。首页滚动电子横幅右下方“以旧换新”标志（如下图红框部分所示），进入“以旧换新”页面。\n\n![以旧换新图片](https://threejs-1251830808.cos.ap-guangzhou.myqcloud.com/73564551948615546537.png)\n\n（以下图片为参考流程，实际以页面为准）\n\n![设备回收图片](https://threejs-1251830808.cos.ap-guangzhou.myqcloud.com/18969746813614796981.png)\n\n选择好您的旧商品后请点击商品图标，进入价格评估页面。\n\n\n\n**二、价格评估**\n\n在价格评估界面上，您需要对旧商品的基本信息（网络制式、存储容量、购买渠道、机身颜色），外观成色（边框背板，屏幕外观，屏幕显示，维修情况），和多功能选项参数进行评估，请如实填写您的手机情况。完成所有参数选择，请估价按钮（如下图红框部分所示）。\n\n完成所有参数选择，请点击“免费询价”按钮（如下图红框部分所示）。\n\n![免费询价图片](https://threejs-1251830808.cos.ap-guangzhou.myqcloud.com/37174946813614947173.png)\n\n**三、价格评估结果**\n\n您可以查看手机的估价和这款手机的回收价格趋势，如果填写有什么问题，可以点击“重新询价”或“重新选择机型”更改；确认无误后，点击“换钱”按钮进入订单页面。\n\n![价格评估结果图片](https://threejs-1251830808.cos.ap-guangzhou.myqcloud.com/28422256813615222482.png)\n\n**四

In [34]:
from langchain.prompts import FewShotPromptTemplate, PromptTemplate
from langchain.prompts.example_selector import MaxMarginalRelevanceExampleSelector, SemanticSimilarityExampleSelector
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings.huggingface import HuggingFaceEmbeddings

embedding_path = "/home/zpc/langchain/blibli/bge-large-zh-v1.5"
embeddings = HuggingFaceEmbeddings(model_name=embedding_path)

vectorStoreDB = FAISS.from_documents(documents=[doc[0], doc1[0], doc3[0]], embedding=embeddings)


vectorStoreDB

In [37]:
vectorStoreDB.similarity_search_with_score("回收手机应该怎么办?")


[(Document(id='4488d4cc-030b-45c9-98be-b31bed0e4427', metadata={'source': '/home/zpc/langchain/blibli/data/yjhx.md'}, page_content='**一、选择回收物品**\n\n点击VMALL您可以在设备回收分类中选择旧商品的类别以及品牌，现在VMALL支持回收部分品牌和型号的手机、笔记本、平板、智能数码以及摄影摄像。如若在此页面上找不到您的品牌或型号，您可以点击品牌栏右下方的加号（如下图红圈部分所示）查看VMALL所支持回收的所有品牌型号。首页滚动电子横幅右下方“以旧换新”标志（如下图红框部分所示），进入“以旧换新”页面。\n\n![以旧换新图片](https://threejs-1251830808.cos.ap-guangzhou.myqcloud.com/73564551948615546537.png)\n\n（以下图片为参考流程，实际以页面为准）\n\n![设备回收图片](https://threejs-1251830808.cos.ap-guangzhou.myqcloud.com/18969746813614796981.png)\n\n选择好您的旧商品后请点击商品图标，进入价格评估页面。\n\n\n\n**二、价格评估**\n\n在价格评估界面上，您需要对旧商品的基本信息（网络制式、存储容量、购买渠道、机身颜色），外观成色（边框背板，屏幕外观，屏幕显示，维修情况），和多功能选项参数进行评估，请如实填写您的手机情况。完成所有参数选择，请估价按钮（如下图红框部分所示）。\n\n完成所有参数选择，请点击“免费询价”按钮（如下图红框部分所示）。\n\n![免费询价图片](https://threejs-1251830808.cos.ap-guangzhou.myqcloud.com/37174946813614947173.png)\n\n**三、价格评估结果**\n\n您可以查看手机的估价和这款手机的回收价格趋势，如果填写有什么问题，可以点击“重新询价”或“重新选择机型”更改；确认无误后，点击“换钱”按钮进入订单页面。\n\n![价格评估结果图片](https://threejs-1251830808.cos.ap-guangzhou.m

In [29]:
import nltk
nltk.download('punkt')



[nltk_data] Downloading package punkt to /home/zpc/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [39]:
from langchain_community.document_loaders import DirectoryLoader
loader = DirectoryLoader("/home/zpc/langchain/blibli/data", show_progress=True)

docs = loader.load()
docs

100%|██████████| 3/3 [00:00<00:00, 27.25it/s]


[Document(metadata={'source': '/home/zpc/langchain/blibli/data/faq_2.txt'}, page_content='众测活动\n\n整体介绍：\n\n一、活动定义：众测是以低价试销的形式，通过收集评价、销量等方法，用于测试市场对新商品的反应，用于及时优化销售策略和引导商家改进。\n\n二、优势：众测价通常比较优惠，以不高于大促促销价为原则，最终以与物权方谈判结果为准。\n\n三、适用范围：华为商城所有产品。\n\n参与方式：\n\n1、华为商城众测的入口在哪里？\n\n华为商城搜索“众测”，即可看到众测入口，点击进入即可；\n\n2、众测上新频次：\n\n众测频道每周一至周五不定期更新上架，部分商品可能会特别调整；\n\n3、众测活动时间：\n\n一款产品众测时间通常为10\n\n20天（热销的商品可能会延期5\n\n10天）。\n\n常见问题：\n\n1、众测商品的来源?\n\n您好，众测商品主要来源为华为商城上架的新品，热销的爆款商品也会不定期通过众测回归上线。\n\n2、众测商品的价格会优惠吗？\n\n您好，众测商品价格对标618和双十一，一般都是该商品某段时间的最低促销价。\n\n3、 众测商品质量会不稳定么？\n\n您好，众测商品也是量产的正品新品，质量与正式上架商品一致。\n\n4、众测商品买下后多久发货？\n\n您好，请以商品页显示为准。\n\n5、提交活动订单后多久内支付？\n\n您好，提交订单后最长付款时效为24小时，逾期订单自动取消\n\n6、成功下单后怎么查询众测订单？\n\n您好，成功下单后，您可以通过华为商城手机APP、华为商城手机WAP版、华为商城电脑网页版任意一端登录下单账户，在“我的订单”查询。（众测商品订单查询方式等同于正常商品订单）\n\n7、订单支付后未发货前可以取消订单吗？\n\n您好，在发货前消费者可以取消订单。\n\n8、取消订单后多久内退回款项？\n\n您好，和华为商城智能家居生态产品通用退款时效一致，3\n\n5个工作日。\n\n9、一个账号可以参与几次众测活动？有限购吗？\n\n您好，一个账号参与众测活动的次数无限制，没有限购。但每次订单只能购买一个商品。如您需要购买多个产品，可以多次参与众测活动。'),
 Document(metadata={'sou

In [41]:
vectorStoreDB = FAISS.from_documents(docs, embedding=embeddings)
vectorStoreDB


In [ ]:
retriver = vectorStoreDB.as_retriever(
    search_type="mmr",
    search_kwargs={"k": 2}
)


information = retriver.get_relevant_documents("请说说回收手机怎么办")
information


/tmp/ipykernel_144345/3465606208.py:7: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  information = retriver.get_relevant_documents("请说说回收手机怎么办")


[Document(id='f0694e70-ff8f-462a-99bb-f4e144eff351', metadata={'source': '/home/zpc/langchain/blibli/data/yjhx.md'}, page_content='一、选择回收物品\n\n点击VMALL您可以在设备回收分类中选择旧商品的类别以及品牌，现在VMALL支持回收部分品牌和型号的手机、笔记本、平板、智能数码以及摄影摄像。如若在此页面上找不到您的品牌或型号，您可以点击品牌栏右下方的加号（如下图红圈部分所示）查看VMALL所支持回收的所有品牌型号。首页滚动电子横幅右下方“以旧换新”标志（如下图红框部分所示），进入“以旧换新”页面。\n\n以旧换新图片\n\n（以下图片为参考流程，实际以页面为准）\n\n设备回收图片\n\n选择好您的旧商品后请点击商品图标，进入价格评估页面。\n\n二、价格评估\n\n在价格评估界面上，您需要对旧商品的基本信息（网络制式、存储容量、购买渠道、机身颜色），外观成色（边框背板，屏幕外观，屏幕显示，维修情况），和多功能选项参数进行评估，请如实填写您的手机情况。完成所有参数选择，请估价按钮（如下图红框部分所示）。\n\n完成所有参数选择，请点击“免费询价”按钮（如下图红框部分所示）。\n\n免费询价图片\n\n三、价格评估结果\n\n您可以查看手机的估价和这款手机的回收价格趋势，如果填写有什么问题，可以点击“重新询价”或“重新选择机型”更改；确认无误后，点击“换钱”按钮进入订单页面。\n\n价格评估结果图片\n\n四、提交订单\n\n在订单页面，您可以从以下两种常规回收方式选择一种进行回收，并提交订单结账。（回收宝支持顺丰上门、上门质检回收，爱回收支持顺丰上门、质检上门、信用回收）\n\n图5.png\n\n（1）顺丰上门回收操作流程\n\n① 回收方式：选择顺丰快递回收\n\n② 提交订单：填写联系人、电话号码、取货地址及上门时间，提交订单\n\n③ 上门取货：回收订单提交成功，快递员会按照填写的上门时间上门取货\n\n④ 确认交易：回收商会在收到旧机的48小时内完成检测，若价格与您提交订单时的价格一致，将立即发放华为代金券。\n\n（2）质检师上门回收操作流程\n\n① 回收方式：选择质检师上门回收。\n\n② 提交订单：填写联系人信息、

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("/home/zpc/langchain/blibli/data/pdf/pdf/Langchain┤є─г╨═AI╙ж╙├╩╡╒╜┐к╖в-╫╩┴╧/01-Langchain/pdf/2312.04005.pdf")
pages = loader.load_and_split()

pages[0]

Document(metadata={'producer': 'pdfTeX-1.40.25', 'creator': 'LaTeX with hyperref', 'creationdate': '2023-12-08T01:57:28+00:00', 'author': '', 'keywords': '', 'moddate': '2023-12-08T01:57:28+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5', 'subject': '', 'title': '', 'trapped': '/False', 'source': '/home/zpc/langchain/blibli/data/pdf/pdf/Langchain┤є─г╨═AI╙ж╙├╩╡╒╜┐к╖в-╫╩┴╧/01-Langchain/pdf/2312.04005.pdf', 'total_pages': 24, 'page': 0, 'page_label': '1'}, page_content='KOALA: Self-Attention Matters in Knowledge Distillation of Latent\nDiffusion Models for Memory-Efficient and Fast Image Synthesis\nYoungwan Lee1,2 Kwanyong Park1 Yoorhim Cho3 Yong-Ju Lee1 Sung Ju Hwang2\n1Electronics and Telecommunications Research Institute (ETRI), South Korea\n2Korea Advanced Institute of Science and Technology (KAIST), South Korea\n3Sookmyung Women’s University, South Korea\nproject page: https://youngwanlee.github.io/KOALA/\nAbstract\n

In [51]:
docs = ""
for item in pages:
    docs += item.page_content
docs


'KOALA: Self-Attention Matters in Knowledge Distillation of Latent\nDiffusion Models for Memory-Efficient and Fast Image Synthesis\nYoungwan Lee1,2 Kwanyong Park1 Yoorhim Cho3 Yong-Ju Lee1 Sung Ju Hwang2\n1Electronics and Telecommunications Research Institute (ETRI), South Korea\n2Korea Advanced Institute of Science and Technology (KAIST), South Korea\n3Sookmyung Women’s University, South Korea\nproject page: https://youngwanlee.github.io/KOALA/\nAbstract\nStable diffusion is the mainstay of the text-to-image (T2I)\nsynthesis in the community due to its generation perfor-\nmance and open-source nature. Recently, Stable Diffusion\nXL (SDXL), the successor of stable diffusion, has received a\nlot of attention due to its significant performance improve-\nments with a higher resolution of 1024 × 1024 and a larger\nmodel. However, its increased computation cost and model\nsize require higher-end hardware (e.g., bigger VRAM GPU)\nfor end-users, incurring higher costs of operation. To ad-\ndr

In [47]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
template = """
'''
{context}
'''
总结上面的论文内容
"""
prompt = ChatPromptTemplate.from_template(template)
output_parser = StrOutputParser()


In [52]:
chain = prompt | model | output_parser
chain.invoke({"context": docs})

'这篇论文《**KOALA: Self-Attention Matters in Knowledge Distillation of Latent Diffusion Models for Memory-Efficient and Fast Image Synthesis**》提出了一种高效的文本到图像（T2I）生成模型KOALA，旨在解决当前最先进的**Stable Diffusion XL (SDXL)** 模型在计算资源和内存上的高昂开销问题。\n\n以下是论文的**核心内容总结**：\n\n---\n\n### 🎯 **研究目标**\n- SDXL虽然质量高（1024×1024分辨率），但模型庞大（U-Net约2.56B参数）、推理慢、需要大显存（如24GB GPU），限制了其在消费级硬件上的部署。\n- **目标**：在**不牺牲太多生成质量**的前提下，构建一个**更小、更快、节省内存的T2I模型**，作为SDXL的高效替代品。\n\n---\n\n### 🔍 **核心方法与创新点**\n\n#### ✅ 1. **深度分析SDXL的U-Net结构**\n- 发现SDXL最耗资源的部分是**U-Net中的Transformer块**，尤其集中在低分辨率特征层（如32×32的特征图）。\n- 这些Transformer块密集分布（尤其在DW-3、Mid、UP-1阶段），占用了绝大多数参数。\n- 传统压缩方法（如BK-SDM）仅简单移除“块对”（residual + transformer），压缩率仅33%，效果有限。\n\n#### ✅ 2. **设计更高效的U-Net结构（KOALA-1B & KOALA-700M）**\n- 基于上述分析，构建了两个轻量级U-Net：\n  - **KOALA-1B**：1.16B参数（比SDXL小54%）\n  - **KOALA-700M**：782M参数（比SDXL小69%，与SDM-v2.0相当）\n- 压缩策略：\n  - **移除部分Transformer块对**（尤其在解码阶段中间层）。\n  - **降低底层Transformer块的深度**（从10个减少到5~6个）。\n  - **在KOALA-700M中彻底移除Mid阶段的Transformer块**（进一步压缩）。\n\n> ✅ **效果**：

In [ ]:
#pypdf in langchain ouput chain
#文本分割
from langchain_community.document_loaders import TextLoader
loader = TextLoader("/home/zpc/langchain/blibli/data/animation.html")
doc = loader.load()
doc

[Document(metadata={'source': '/home/zpc/langchain/blibli/data/animation.html'}, page_content='<!DOCTYPE html>\n<html lang="zh">\n\t<head>\n\t\t<meta charset="utf-8" />\n\t\t<base href="../../../" />\n\t\t<script src="page.js"></script>\n\t\t<link type="text/css" rel="stylesheet" href="page.css" />\n\t</head>\n\t<body>\n\t\t<h1>动画系统（[name]）</h1>\n\n\t\t<h2>概述</h2>\n\n\t\t<p class="desc">\n\t\t\t在three.js动画系统中，您可以为模型的各种属性设置动画：\n\t\t\t[page:SkinnedMesh]（蒙皮和装配模型）的骨骼，morph targets（变形目标），\n\t\t\t不同的材料属性（颜色，不透明度，布尔运算），可见性和变换。动画属性可以淡入、淡出、交叉淡化和扭曲。\n\t\t\t在相同或不同物体上同时发生的动画的权重和时间比例的变化可以独立地进行。\n\t\t\t相同或不同物体的动画也可以同步发生。\n\t\t\t<br /><br />\n\t\t\t为了在一个同构系统中实现所有这一切，\n\t\t\tthree.js的动画系统[link:https://github.com/mrdoob/three.js/issues/6881 在2015年彻底改变]（注意过时的信息！），\n\t\t\t它现在有一个与Unity/虚幻4引擎类似的架构。此页面简要阐述了这个系统中的主要组件以及它们如何协同工作。\n\t\t</p>\n\n\t\t<h3>动画片段（Animation Clips）</h3>\n\n\t\t<p class="desc">\n\t\t\t如果您已成功导入3D动画对象（无论它是否有骨骼或变形目标或两者皆有都不要紧）——\n\t\t\t例如使用[link:https://github.com/KhronosGroup/glTF-Blender-

In [ ]:
headers_to_split_on = [
    ("h1", "Header1"),
    ("h2", "Header2"),
    ("h3", "Header3")
]



In [56]:
from langchain_text_splitters import HTMLHeaderTextSplitter
html_splitter = HTMLHeaderTextSplitter(headers_to_split_on)

html_header_splits = html_splitter.split_text(doc[0].page_content)
html_header_splits

[Document(metadata={'Header1': '动画系统（[name]）'}, page_content='动画系统（[name]）'),
 Document(metadata={'Header1': '动画系统（[name]）', 'Header2': '概述'}, page_content='概述'),
 Document(metadata={'Header1': '动画系统（[name]）', 'Header2': '概述'}, page_content='在three.js动画系统中，您可以为模型的各种属性设置动画：\n\t\t\t[page:SkinnedMesh]（蒙皮和装配模型）的骨骼，morph targets（变形目标），\n\t\t\t不同的材料属性（颜色，不透明度，布尔运算），可见性和变换。动画属性可以淡入、淡出、交叉淡化和扭曲。\n\t\t\t在相同或不同物体上同时发生的动画的权重和时间比例的变化可以独立地进行。\n\t\t\t相同或不同物体的动画也可以同步发生。 为了在一个同构系统中实现所有这一切，\n\t\t\tthree.js的动画系统[link:https://github.com/mrdoob/three.js/issues/6881 在2015年彻底改变]（注意过时的信息！），\n\t\t\t它现在有一个与Unity/虚幻4引擎类似的架构。此页面简要阐述了这个系统中的主要组件以及它们如何协同工作。'),
 Document(metadata={'Header1': '动画系统（[name]）', 'Header2': '概述', 'Header3': '动画片段（Animation Clips）'}, page_content='动画片段（Animation Clips）'),
 Document(metadata={'Header1': '动画系统（[name]）', 'Header2': '概述', 'Header3': '动画片段（Animation Clips）'}, page_content='如果您已成功导入3D动画对象（无论它是否有骨骼或变形目标或两者皆有都不要紧）——\n\t\t\t例如使用[link:https://github.com/KhronosGroup/glTF-Blender-IO g

In [58]:
from langchain_community.vectorstores import Chroma
from langchain_core.documents import Document

docs = [
    Document(
        page_content="一群科学家带回恐龙爆发了混乱",
        metadata={"year": 1993, "rating": 7.7, "genre": "科幻小说"},
    ),
    Document(
        page_content="故事发生在1920年北洋年间中国南方，马邦德花钱买官，购得“萨南康省”的县长一职，坐“马拉的火车”赴任途中遭马匪张麻子一行人伏击",
        metadata={"year": 2010, "director": "姜文", "rating": 8.2},
    ),
    Document(
        page_content="话说孙悟空护送唐三藏前往西天取经，半路却和牛魔王合谋要杀害唐三藏，并偷走了紫霞仙子持有的月光宝盒。观音闻讯赶到，欲除掉孙悟空以免危害苍生。唐三藏慈悲为怀，愿意一命赔一命，感化劣徒，观音遂令孙悟空五百年后投胎做人，赎其罪孽。",
        metadata={"year": 1994, "director": "刘镇伟", "rating": 8.6},
    ),
    Document(
        page_content="故事背景设定在2075年，讲述了太阳即将毁灭，毁灭之后的太阳系已经不适合人类生存，而面对绝境，人类将开启“流浪地球”计划，试图带着地球一起逃离太阳系，寻找人类新家园的故事。",
        metadata={"year": 2019, "director": "郭帆", "rating": 8.3},
    ),
    Document(
        page_content="该片讲述了耿浩和好哥们郝义一场荒诞而有趣的“寻爱之旅”。该片采用双线叙事的手法，以耿浩和康小雨婚姻破裂为叙事的起点，在郝义携耿浩前往剧组送道具途中“寻爱”的故事中，穿插着昔日康小雨孤身前往大理并与耿浩相遇的前尘往事，讲述着在现代生活中不同人群对婚姻、生活与理想的不同追求。",
        metadata={"year": 2014, "genre": "喜剧"},
    ),
]

In [ ]:
vectorstore = Chroma.from_documents(docs, embeddings)


In [61]:
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.retrievers.self_query.base import SelfQueryRetriever

metadata_field_info = [
    AttributeInfo(
        name="genre",
        description='电影的类型。 ["科幻小说","喜剧","剧情片"，"惊悚片","爱情片","动作片","动画片"]之一',
        type="string",
    ),
    AttributeInfo(
        name="year",
        description="电影上映年份",
        type="integer",
    ),
    AttributeInfo(
        name="director",
        description="电影导演的名字",
        type="string",
    ),
    AttributeInfo(
        name="rating", description="电影评分为 1-10", type="float"
    ),
]


document_content_description = "电影的简要概述"

In [62]:
retriever = SelfQueryRetriever.from_llm(
    model,
    vectorstore,
    document_content_description,
    metadata_field_info,
    enable_limit=True,
    search_kwargs={"k": 2}
)

In [63]:
# This example only specifies a filter
retriever.invoke("给我推荐一部评分8.5以上的电影")

[Document(metadata={'director': '刘镇伟', 'year': 1994, 'rating': 8.6}, page_content='话说孙悟空护送唐三藏前往西天取经，半路却和牛魔王合谋要杀害唐三藏，并偷走了紫霞仙子持有的月光宝盒。观音闻讯赶到，欲除掉孙悟空以免危害苍生。唐三藏慈悲为怀，愿意一命赔一命，感化劣徒，观音遂令孙悟空五百年后投胎做人，赎其罪孽。')]